In [2]:
import os.path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

### Получаем необходимый путь

In [3]:
file_path = os.getcwd()
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(file_path)))

In [151]:
SHARED_DIR = os.path.join(BASE_DIR, 'shared')
HOMEWORKS_DIR = os.path.join(SHARED_DIR, 'homeworks')
PYTHON_MINIPROJECTS_DIR = os.path.join(HOMEWORKS_DIR, 'python_ds_miniprojects')
MINIPROJECT_EXTRA_5_DIR = os.path.join(PYTHON_MINIPROJECTS_DIR, '5_subsid/subsid')

In [152]:
MINIPROJECT_EXTRA_5_DIR

'/mnt/HC_Volume_18315164/home-jupyter/jupyter-d-barkov/shared/homeworks/python_ds_miniprojects/5_subsid/subsid'

In [153]:
os.listdir(MINIPROJECT_EXTRA_5_DIR)

['prod_activations_logs.csv',
 'tm_sales_1.csv',
 'tm_sales_2.csv',
 'tm_sales_3.csv']

### Осуществляем присвоение имени переменных через globals() и назначаем им соответствующие датафреймы:

In [154]:
df_dict = {}
for file in os.listdir(MINIPROJECT_EXTRA_5_DIR):
    path = os.path.join(MINIPROJECT_EXTRA_5_DIR, file)
    df_name = f"{file.split('.')[0]}_df"
    df_dict[df_name] = pd.read_csv(path, sep=';')

In [155]:
df_dict.keys()
df_dict['tm_sales_3_df'].dtypes

FILIAL_ID     int64
PROD_ID       int64
SUBS_ID      object
ACT_DTTM     object
dtype: object

In [124]:
prod_activations_logs_df

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
1,id4651830,1954,2020-03-20 14:59:00,2020-01-12 00:00:00
5,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00
2,id7461794,3310,2020-03-20 17:25:00,2020-01-12 00:00:00
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00
4,id8238421,1859,2020-01-03 11:42:00,2020-01-03 11:43:00
3,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00
2,id2185490,3210,2020-03-16 16:28:00,2020-01-12 00:00:00
6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00
2,id8741631,5677,2020-03-19 12:28:00,2020-01-12 00:00:00


In [42]:
tm_sales_1_df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25


In [43]:
tm_sales_2_df

,FILIAL_ID,SUBS_ID,PROD_ID,ACT_DTTM
0,3,id5416547,1743,17-03-2020 10:17
1,4,NaN,7421,12-03-2020 11:25
2,7,8362218,9879,05-03-2020 11:42
3,2,2185490,3210,16-03-2020 16:28


In [44]:
tm_sales_3_df

,FILIAL_ID,PROD_ID,SUBS_ID,ACT_DTTM
0,3,1499,id5764122,18-03-2020 15:44
1,6,3020,id7642700,15-03-2020 14:21
2,2,5677,id1374509,17-03-2020 11:48


### Алгоритм проверки правильности написания данных в колонке SUBS_ID:

In [156]:
for df_name in df_dict.keys():
    
    # remove NaN data
    df_dict[df_name] = df_dict[df_name].dropna()
    
    # making SUBS_ID column data in propriate type (idxxx) by adding 'id' where it is absent:
    df_dict[df_name]['SUBS_ID'] = df_dict[df_name]['SUBS_ID'].apply(lambda x: f'id{x}' if not x.startswith('id') else x)

In [157]:
# checking reverse
df_dict['tm_sales_2_df']

,FILIAL_ID,SUBS_ID,PROD_ID,ACT_DTTM
0,3,id5416547,1743,17-03-2020 10:17
2,7,id8362218,9879,05-03-2020 11:42
3,2,id2185490,3210,16-03-2020 16:28


### Отсеивание из prod_activations_logs_df строк данных, где  (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)

In [158]:
df_dict['prod_activations_logs_df'].dtypes

SUBS_ID       object
PROD_ID        int64
START_DTTM    object
END_DTTM      object
dtype: object

##### Перевод данных в START_DTTM и END_DTTM к типу datetime64 

In [159]:
df_dict['prod_activations_logs_df'].START_DTTM, df_dict['prod_activations_logs_df'].END_DTTM = pd.to_datetime(df_dict['prod_activations_logs_df'].START_DTTM, format="%d-%m-%Y %H:%M"), pd.to_datetime(df_dict['prod_activations_logs_df'].END_DTTM, format="%d-%m-%Y %H:%M")

In [160]:
df_dict['prod_activations_logs_df'].dtypes

SUBS_ID               object
PROD_ID                int64
START_DTTM    datetime64[ns]
END_DTTM      datetime64[ns]
dtype: object

##### Создание колонки timedelta, как разницы END_DTTM и START_DTTM:

In [161]:
df_dict['prod_activations_logs_df']['timedelta_minutes'] = df_dict['prod_activations_logs_df'].END_DTTM - df_dict['prod_activations_logs_df'].START_DTTM

In [162]:
df_dict['prod_activations_logs_df']

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,timedelta_minutes
1,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
5,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00
2,id7461794,3310,2020-03-20 17:25:00,2020-12-01 00:00:00,255 days 06:35:00
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
4,id8238421,1859,2020-03-01 11:42:00,2020-03-01 11:43:00,0 days 00:01:00
3,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00,32 days 00:00:00
2,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00
2,id8741631,5677,2020-03-19 12:28:00,2020-12-01 00:00:00,256 days 11:32:00


##### Перевод времени в timedelta в минуты

In [163]:
df_dict['prod_activations_logs_df']['timedelta_minutes'] = df_dict['prod_activations_logs_df']['timedelta_minutes'].apply(lambda x: x.total_seconds() / 60)

In [164]:
df_dict['prod_activations_logs_df']

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,timedelta_minutes
1,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,367741.0
5,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,3.0
2,id7461794,3310,2020-03-20 17:25:00,2020-12-01 00:00:00,367595.0
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,11563.0
4,id8238421,1859,2020-03-01 11:42:00,2020-03-01 11:43:00,1.0
3,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00,46080.0
2,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,373412.0
6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,561.0
2,id8741631,5677,2020-03-19 12:28:00,2020-12-01 00:00:00,369332.0


##### Отсеиваем незасчитанные продажи

In [165]:
df_dict['prod_activations_logs_df'] = df_dict['prod_activations_logs_df'][df_dict['prod_activations_logs_df'].timedelta_minutes > 5]

In [166]:
df_dict['prod_activations_logs_df']

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,timedelta_minutes
1,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,367741.0
2,id7461794,3310,2020-03-20 17:25:00,2020-12-01 00:00:00,367595.0
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,11563.0
3,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00,46080.0
2,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,373412.0
6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,561.0
2,id8741631,5677,2020-03-19 12:28:00,2020-12-01 00:00:00,369332.0


### Составляем общий датафрейм

In [167]:
df_dict['prod_activations_logs_df'].columns.shape[0]

5

In [140]:
counter = 0
for key in df_dict.keys():
    print(f'key - {key}')
    if counter == 0:
        general_df = df_dict[key]
        counter = 1
        continue
    print(f'current_df: {df_dict[key].columns}', f'general_df: {general_df.columns}', sep='\n')   
    # Comparing types of df throw columns, in case they are equal - making concantenation:
    if df_dict[key].columns.str == general_df.columns.str:
        general_df = pd.concat([general_df, df_dict[key]])
        print('done concat')
    else:
        general_df.merge(df_dict[key], how='inner', on='SUBS_ID')
        print('done merge')

key - prod_activations_logs_df
key - tm_sales_1_df
current_df: Index(['SUBS_ID', 'FILIAL_ID', 'PROD_ID', 'ACT_DTTM'], dtype='object')
general_df: Index(['SUBS_ID', 'PROD_ID', 'START_DTTM', 'END_DTTM', 'timedelta_minutes'], dtype='object')
done merge
key - tm_sales_2_df
current_df: Index(['FILIAL_ID', 'SUBS_ID', 'PROD_ID', 'ACT_DTTM'], dtype='object')
general_df: Index(['SUBS_ID', 'PROD_ID', 'START_DTTM', 'END_DTTM', 'timedelta_minutes'], dtype='object')
done merge
key - tm_sales_3_df
current_df: Index(['FILIAL_ID', 'PROD_ID', 'SUBS_ID', 'ACT_DTTM'], dtype='object')
general_df: Index(['SUBS_ID', 'PROD_ID', 'START_DTTM', 'END_DTTM', 'timedelta_minutes'], dtype='object')
done merge


In [141]:
general_df

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,timedelta_minutes
1,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,367741.0
2,id7461794,3310,2020-03-20 17:25:00,2020-12-01 00:00:00,367595.0
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,11563.0
3,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00,46080.0
2,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,373412.0
6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,561.0
2,id8741631,5677,2020-03-19 12:28:00,2020-12-01 00:00:00,369332.0


In [117]:
', '.join([i for i in general_df.SUBS_ID.sort_values()])

'id2185490, id4651830, id5416547, id7461794, id7642700, id8641743, id8741631'

In [177]:
counter = 0
for key in df_dict.keys():
    print(f'key - {key}')
    if df_dict[key].columns.shape[0] > 4:
        df_to_merge = df_dict[key]
        continue
    elif counter == 0:
        general_df = df_dict[key]
        counter = 1
        print(f'done general df as {key}')
    else:
        general_df = pd.concat([general_df, df_dict[key]])
        print(f'done concat between general and {key}')
        
general_df = df_to_merge.merge(general_df, how='inner', on=['SUBS_ID', 'PROD_ID'])
print('done merge')

key - prod_activations_logs_df
key - tm_sales_1_df
done general df as tm_sales_1_df
key - tm_sales_2_df
done concat between general and tm_sales_2_df
key - tm_sales_3_df
done concat between general and tm_sales_3_df
done merge


In [178]:
general_df

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,timedelta_minutes,FILIAL_ID,ACT_DTTM
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,367741.0,1,20-03-2020 14:59
1,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,11563.0,3,17-03-2020 10:17
2,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,373412.0,2,16-03-2020 16:28
3,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,561.0,6,15-03-2020 14:21


In [179]:
', '.join([i for i in general_df.SUBS_ID.sort_values()])

'id2185490, id4651830, id5416547, id7642700'